In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lstm = nn.LSTM(3, 3)

In [4]:
inputs = [torch.randn(1, 3) for _ in range(5)]

In [5]:
inputs

[tensor([[-0.5525,  0.6355, -0.3968]]),
 tensor([[-0.6571, -1.6428,  0.9803]]),
 tensor([[-0.0421, -0.8206,  0.3133]]),
 tensor([[-1.1352,  0.3773, -0.2824]]),
 tensor([[-2.5667, -1.4303,  0.5009]])]

In [8]:
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))

In [9]:
hidden

(tensor([[[ 0.5438, -0.4057,  1.1341]]]),
 tensor([[[-1.1115,  0.3501, -0.7703]]]))

In [11]:
for i in inputs:
    out, hidden = lstm(i.view(1, 1, -1), hidden)

In [12]:
out

tensor([[[-0.3600,  0.0893,  0.0215]]])

In [13]:
hidden

(tensor([[[-0.3600,  0.0893,  0.0215]]]),
 tensor([[[-1.1298,  0.4467,  0.0254]]]))

In [14]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)

In [15]:
inputs

tensor([[[-0.5525,  0.6355, -0.3968]],

        [[-0.6571, -1.6428,  0.9803]],

        [[-0.0421, -0.8206,  0.3133]],

        [[-1.1352,  0.3773, -0.2824]],

        [[-2.5667, -1.4303,  0.5009]]])

In [16]:
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))

In [17]:
out, hidden = lstm(inputs, hidden)

In [18]:
out

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]])

In [19]:
hidden

(tensor([[[-0.3368,  0.0959, -0.0538]]]),
 tensor([[[-0.9825,  0.4715, -0.0633]]]))

In [20]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [21]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]

In [22]:
word_to_ix = {}

In [23]:
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [24]:
print(word_to_ix)

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [25]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

In [26]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [31]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_dim), torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores